<a href="https://colab.research.google.com/github/indranildchandra/JD_Keywords_Extractor/blob/master/src/Topic_Modeling_on_Job_Descriptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

#upload online-job-posts.zip file for training

In [68]:
!unzip online-job-posts.zip

Archive:  online-job-posts.zip
replace online-job-posts.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: online-job-posts.csv    


In [69]:
!ls -ltr

total 2651456
-rw-r--r-- 1 root root   95435519 Apr  3 03:12 online-job-posts.csv
drwxr-xr-x 1 root root       4096 Apr  4 20:20 sample_data
-rw-r--r-- 1 root root  279960406 Apr  7 18:41 test.csv
-rw-r--r-- 1 root root 1251803869 Apr  7 18:41 train.csv
-rw-r--r-- 1 root root  319129989 Apr  7 18:41 user_features.csv
-rw-r--r-- 1 root root   13254376 Apr 22 16:43 online-job-posts.zip
-rw-r--r-- 1 root root  713029252 Apr 22 20:02 dataset.zip
-rw-r--r-- 1 root root    2316149 Apr 22 20:30 job_description_bow_corpus.pkl
-rw-r--r-- 1 root root    2607134 Apr 22 20:30 job_description_tfidf_corpus.pkl
-rw-r--r-- 1 root root     189635 Apr 22 20:30 job_description_dictionary.gensim
-rw-r--r-- 1 root root    5715308 Apr 22 20:30 job_requirement_bow_corpus.pkl
-rw-r--r-- 1 root root    6098902 Apr 22 20:30 job_requirement_tfidf_corpus.pkl
-rw-r--r-- 1 root root     251569 Apr 22 20:30 job_requirement_dictionary.gensim
-rw-r--r-- 1 root root    5714777 Apr 22 20:30 required_qualification_bow_co

In [23]:
!pip install pyLDAvis
!pip install paramiko

In [0]:
import spacy
import nltk
import random
import pickle
import gensim
import pandas as pd
import pyLDAvis.gensim

from spacy.lang.en import English
from spacy import displacy
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models

In [0]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=PendingDeprecationWarning)
warnings.filterwarnings('ignore', category=ResourceWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [26]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
parser = spacy.load('en')

In [0]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.is_punct:
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('at')
        elif token.pos_ == "ADJ" or token.pos_ == "VERB" or token.pos_ == "RBR" or token.pos_ == "RBS" or token.pos_ == "RB" or token.pos_ == "RP":
            continue
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [0]:
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [30]:
for w in ['ran', 'happier', 'charging']:
    print(w, get_lemma(w), get_lemma2(w))

ran run ran
happier happy happier
charging charge charging


In [0]:
en_stop = set(nltk.corpus.stopwords.words('english'))

In [32]:
#[["jobpost","date","Title","Company","AnnouncementCode","Term","Eligibility","Audience","StartDate","Duration","Location","JobDescription","JobRequirement","RequiredQual","Salary","ApplicationP","OpeningDate","Deadline","Notes","AboutC","Attach","Year","Month","IT"]]
raw_data = pd.read_csv("online-job-posts.csv")
raw_data.fillna("", inplace = True)

print(raw_data.head(2))

                                             jobpost         date  \
0  AMERIA Investment Consulting Company\r\nJOB TI...  Jan 5, 2004   
1  International Research & Exchanges Board (IREX...  Jan 7, 2004   

                                               Title  \
0                            Chief Financial Officer   
1  Full-time Community Connections Intern (paid i...   

                                           Company AnnouncementCode Term  \
0             AMERIA Investment Consulting Company                         
1  International Research & Exchanges Board (IREX)                         

  Eligibility Audience StartDate  Duration  ...   Salary  \
0                                           ...            
1                                 3 months  ...            

                                        ApplicationP OpeningDate   Deadline  \
0  To apply for this position, please submit a\r\...              26-Jan-04   
1  Please submit a cover letter and resume to:\r\...   

In [0]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [0]:
job_description_data = []
job_requirement_data = []
required_qualification_data = []

for index, entry in raw_data.iterrows():
  job_description_tokens = prepare_text_for_lda(str(entry["JobDescription"]).strip())
  job_requirement_tokens = prepare_text_for_lda(str(entry["JobRequirement"]).strip())
  required_qualification_tokens = prepare_text_for_lda(str(entry["RequiredQual"]).strip())
  
  if not len(job_description_tokens) == 0:
    job_description_data.append(job_description_tokens)
  if not len(job_requirement_tokens) == 0:
    job_requirement_data.append(job_requirement_tokens)
  if not len(required_qualification_tokens) == 0:
    required_qualification_data.append(required_qualification_tokens)

In [0]:
job_description_dictionary = corpora.Dictionary(job_description_data)
job_requirement_dictionary = corpora.Dictionary(job_requirement_data)
required_qualification_dictionary = corpora.Dictionary(required_qualification_data)

In [0]:
job_description_bow_corpus = [job_description_dictionary.doc2bow(text) for text in job_description_data]
job_requirement_bow_corpus = [job_requirement_dictionary.doc2bow(text) for text in job_requirement_data]
required_qualification_bow_corpus = [required_qualification_dictionary.doc2bow(text) for text in required_qualification_data]

In [0]:
job_description_tfidf_model = models.TfidfModel(job_description_bow_corpus)
job_description_tfidf_corpus = job_description_tfidf_model[job_description_bow_corpus]

job_requirement_tfidf_model = models.TfidfModel(job_requirement_bow_corpus)
job_requirement_tfidf_corpus = job_requirement_tfidf_model[job_requirement_bow_corpus]

required_qualification_tfidf_model = models.TfidfModel(required_qualification_bow_corpus)
required_qualification_tfidf_corpus = required_qualification_tfidf_model[required_qualification_bow_corpus]

In [0]:
pickle.dump(job_description_bow_corpus, open('job_description_bow_corpus.pkl', 'wb'))
pickle.dump(job_description_tfidf_corpus, open('job_description_tfidf_corpus.pkl', 'wb'))
job_description_dictionary.save('job_description_dictionary.gensim')

pickle.dump(job_requirement_bow_corpus, open('job_requirement_bow_corpus.pkl', 'wb'))
pickle.dump(job_requirement_tfidf_corpus, open('job_requirement_tfidf_corpus.pkl', 'wb'))
job_requirement_dictionary.save('job_requirement_dictionary.gensim')

pickle.dump(required_qualification_bow_corpus, open('required_qualification_bow_corpus.pkl', 'wb'))
pickle.dump(required_qualification_tfidf_corpus, open('required_qualification_tfidf_corpus.pkl', 'wb'))
required_qualification_dictionary.save('required_qualification_dictionary.gensim')

In [39]:
!grep -c ^processor /proc/cpuinfo
# Set this value to NUM_OF_CORES

2


In [0]:
NUM_OF_CORES = 2

In [0]:
JOB_DESCRIPTION_NUM_TOPICS = 20
job_description_bow_lda_model = gensim.models.LdaMulticore(job_description_bow_corpus, num_topics = JOB_DESCRIPTION_NUM_TOPICS, id2word=job_description_dictionary, passes=100, workers=NUM_OF_CORES)
job_description_bow_lda_model.save('job_description_bow_lda_model.gensim')
job_description_tfidf_lda_model = gensim.models.LdaMulticore(job_description_tfidf_corpus, num_topics = JOB_DESCRIPTION_NUM_TOPICS, id2word=job_description_dictionary, passes=100, workers=NUM_OF_CORES)
job_description_tfidf_lda_model.save('job_description_tfidf_lda_model.gensim')

In [0]:
JOB_REQUIREMENT_NUM_TOPICS = 20
job_requirement_bow_lda_model = gensim.models.LdaMulticore(job_requirement_bow_corpus, num_topics = JOB_REQUIREMENT_NUM_TOPICS, id2word=job_requirement_dictionary, passes=100, workers=NUM_OF_CORES)
job_requirement_bow_lda_model.save('job_requirement_bow_lda_model.gensim')
job_requirement_tfidf_lda_model = gensim.models.LdaMulticore(job_requirement_tfidf_corpus, num_topics = JOB_REQUIREMENT_NUM_TOPICS, id2word=job_requirement_dictionary, passes=100, workers=NUM_OF_CORES)
job_requirement_tfidf_lda_model.save('job_requirement_tfidf_lda_model.gensim')

In [0]:
REQUIRED_QUALIFICATION_NUM_TOPICS = 20
required_qualification_bow_lda_model = gensim.models.LdaMulticore(required_qualification_bow_corpus, num_topics = REQUIRED_QUALIFICATION_NUM_TOPICS, id2word=required_qualification_dictionary, passes=100, workers=NUM_OF_CORES)
required_qualification_bow_lda_model.save('required_qualification_bow_lda_model.gensim')
required_qualification_tfidf_lda_model = gensim.models.LdaMulticore(required_qualification_tfidf_corpus, num_topics = REQUIRED_QUALIFICATION_NUM_TOPICS, id2word=required_qualification_dictionary, passes=100, workers=NUM_OF_CORES)
required_qualification_tfidf_lda_model.save('required_qualification_tfidf_lda_model.gensim')

In [46]:
job_description_bow_topics = job_description_bow_lda_model.print_topics(num_words=10)
print("Job Description Topics (BoW): ")
for topic in job_description_bow_topics:
    print(topic)

job_description_tfidf_topics = job_description_tfidf_lda_model.print_topics(num_words=10)
print("Job Description Topics (TF-IDF): ")
for topic in job_description_tfidf_topics:
    print(topic)

Job Description Topics (BoW): 
(0, '0.067*"credit" + 0.039*"loan" + 0.033*"resource" + 0.023*"specialist" + 0.023*"officer" + 0.023*"employee" + 0.022*"monitoring" + 0.019*"procedure" + 0.019*"human" + 0.017*"lending"')
(1, '0.040*"consultant" + 0.035*"project" + 0.028*"team" + 0.025*"support" + 0.019*"investment" + 0.019*"management" + 0.018*"armenia" + 0.018*"client" + 0.017*"contract" + 0.016*"europe"')
(2, '0.045*"development" + 0.042*"developer" + 0.042*"company" + 0.041*"team" + 0.034*"application" + 0.033*"experience" + 0.029*"software" + 0.027*"candidate" + 0.021*"user" + 0.021*"highly"')
(3, '0.082*"company" + 0.071*"marketing" + 0.050*"sales" + 0.041*"incumbent" + 0.038*"manager" + 0.030*"product" + 0.027*"strategy" + 0.025*"market" + 0.019*"development" + 0.018*"business"')
(4, '0.113*"project" + 0.043*"development" + 0.037*"implementation" + 0.028*"activity" + 0.023*"manager" + 0.021*"community" + 0.020*"management" + 0.019*"program" + 0.017*"construction" + 0.016*"incumben

In [47]:
job_requirement_bow_topics = job_requirement_bow_lda_model.print_topics(num_words=10)
print("\nJob Requirement Topics (BoW): ")
for topic in job_requirement_bow_topics:
    print(topic)

job_requirement_tfidf_topics = job_requirement_tfidf_lda_model.print_topics(num_words=10)
print("\nJob Requirement Topics (TF-IDF): ")
for topic in job_requirement_tfidf_topics:
    print(topic)


Job Requirement Topics (BoW): 
(0, '0.073*"system" + 0.034*"network" + 0.027*"security" + 0.027*"equipment" + 0.027*"software" + 0.023*"database" + 0.023*"support" + 0.021*"maintenance" + 0.016*"server" + 0.015*"user"')
(1, '0.041*"accounting" + 0.040*"report" + 0.030*"cash" + 0.024*"payment" + 0.022*"account" + 0.018*"prepare" + 0.018*"transactions" + 0.017*"bank" + 0.017*"company" + 0.014*"system"')
(2, '0.050*"client" + 0.049*"credit" + 0.042*"loan" + 0.040*"bank" + 0.017*"banks" + 0.016*"document" + 0.014*"process" + 0.013*"customer" + 0.012*"prepare" + 0.012*"business"')
(3, '0.045*"management" + 0.028*"control" + 0.025*"business" + 0.024*"audit" + 0.020*"system" + 0.020*"analysis" + 0.019*"risk" + 0.018*"procedure" + 0.017*"process" + 0.017*"budget"')
(4, '0.165*"customer" + 0.042*"sales" + 0.037*"service" + 0.031*"product" + 0.028*"company" + 0.024*"services" + 0.019*"information" + 0.012*"need" + 0.012*"client" + 0.011*"support"')
(5, '0.067*"data" + 0.027*"report" + 0.026*"an

In [48]:
required_qualification_bow_topics = required_qualification_bow_lda_model.print_topics(num_words=10)
print("\nRequired Qualification Topics (BoW): ")
for topic in required_qualification_bow_topics:
    print(topic)

required_qualification_tfidf_topics = required_qualification_tfidf_lda_model.print_topics(num_words=10)
print("\nRequired Qualification Topics (TF-IDF): ")
for topic in required_qualification_tfidf_topics:
    print(topic)


Required Qualification Topics (BoW): 
(0, '0.089*"knowledge" + 0.048*"system" + 0.047*"experience" + 0.042*"network" + 0.030*"windows" + 0.022*"server" + 0.021*"administration" + 0.020*"language" + 0.018*"technology" + 0.016*"ability"')
(1, '0.061*"knowledge" + 0.061*"experience" + 0.037*"skill" + 0.030*"ability" + 0.029*"software" + 0.025*"testing" + 0.024*"development" + 0.017*"plus" + 0.017*"tool" + 0.015*"years"')
(2, '0.096*"knowledge" + 0.062*"accounting" + 0.039*"finance" + 0.038*"experience" + 0.034*"skill" + 0.027*"language" + 0.026*"banking" + 0.024*"years" + 0.024*"degree" + 0.023*"ability"')
(3, '0.097*"experience" + 0.065*"knowledge" + 0.032*"development" + 0.031*"years" + 0.030*"work" + 0.027*".net" + 0.026*"skill" + 0.026*"framework" + 0.025*"language" + 0.024*"server"')
(4, '0.094*"knowledge" + 0.089*"experience" + 0.066*"language" + 0.050*"work" + 0.047*"years" + 0.036*"field" + 0.035*"license" + 0.034*"plus" + 0.034*"degree" + 0.031*"education"')
(5, '0.071*"knowledg

In [50]:
new_doc = "Investment Consulting Company is seeking a Chief Financial Officer. This position manages the company's fiscal and administrative functions, provides highly responsible and technically complex staff assistance to the Executive Director. The work performed requires a high level of technical proficiency in financial management and investment management, as well as management, supervisory and administrative skills."
document = parser(new_doc)

print("Noun Phrases...")
print("--- [Format: Noun Phrase -> Root Text] ---")
for noun_phrase in document.noun_chunks:
  #print(noun_phrase.text, noun_phrase.label_, noun_phrase.root.text)
  print(noun_phrase.text + " -> " + noun_phrase.root.text)

new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = job_description_dictionary.doc2bow(new_doc)
new_doc_tfidf = job_description_tfidf_model[new_doc_bow]

# print("\nTopics relevant to new document are: ")
# print(job_description_bow_lda_model.get_document_topics(new_doc_bow, minimum_probability=0.1))

print("\n\nTopics relevant to new document are (BoW): ")
for index, score in sorted(job_description_bow_lda_model[new_doc_bow], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, job_description_bow_lda_model.print_topic(index, 10)))
 
print("\n\nTopics relevant to new document are (TF-IDF): ")
for index, score in sorted(job_description_tfidf_lda_model[new_doc_tfidf], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, job_description_tfidf_lda_model.print_topic(index, 10)))

Noun Phrases...
--- [Format: Noun Phrase -> Root Text] ---
Investment Consulting Company -> Company
a Chief Financial Officer -> Officer
This position -> position
the company's fiscal and administrative functions -> functions
highly responsible and technically complex staff assistance -> assistance
the Executive Director -> Director
The work -> work
a high level -> level
technical proficiency -> proficiency
financial management and investment management -> management
management -> management
skills -> skills


Topics relevant to new document are (BoW): 
Score: 0.6444883942604065	 Topic: 0.034*"office" + 0.028*"accounting" + 0.027*"management" + 0.027*"company" + 0.025*"director" + 0.024*"chief" + 0.023*"supervision" + 0.023*"incumbent" + 0.022*"accountant" + 0.021*"activity"
Score: 0.15489837527275085	 Topic: 0.040*"consultant" + 0.035*"project" + 0.028*"team" + 0.025*"support" + 0.019*"investment" + 0.019*"management" + 0.018*"armenia" + 0.018*"client" + 0.017*"contract" + 0.016*"euro

In [0]:
job_description_dictionary = gensim.corpora.Dictionary.load('job_description_dictionary.gensim')
job_description_bow_corpus = pickle.load(open('job_description_bow_corpus.pkl', 'rb'))
job_description_bow_lda_model = gensim.models.ldamodel.LdaModel.load('job_description_bow_lda_model.gensim')
job_description_tfidf_corpus = pickle.load(open('job_description_tfidf_corpus.pkl', 'rb'))
job_description_tfidf_lda_model = gensim.models.ldamodel.LdaModel.load('job_description_tfidf_lda_model.gensim')

job_requirement_dictionary = gensim.corpora.Dictionary.load('job_requirement_dictionary.gensim')
job_requirement_bow_corpus = pickle.load(open('job_requirement_bow_corpus.pkl', 'rb'))
job_requirement_bow_lda_model = gensim.models.ldamodel.LdaModel.load('job_requirement_bow_lda_model.gensim')
job_requirement_tfidf_corpus = pickle.load(open('job_requirement_tfidf_corpus.pkl', 'rb'))
job_requirement_tfidf_lda_model = gensim.models.ldamodel.LdaModel.load('job_requirement_tfidf_lda_model.gensim')

required_qualification_dictionary = gensim.corpora.Dictionary.load('required_qualification_dictionary.gensim')
required_qualification_bow_corpus = pickle.load(open('required_qualification_bow_corpus.pkl', 'rb'))
required_qualification_bow_lda_model = gensim.models.ldamodel.LdaModel.load('required_qualification_bow_lda_model.gensim')
required_qualification_tfidf_corpus = pickle.load(open('required_qualification_tfidf_corpus.pkl', 'rb'))
required_qualification_tfidf_lda_model = gensim.models.ldamodel.LdaModel.load('required_qualification_tfidf_lda_model.gensim')

In [52]:
job_description_bow_lda_display = pyLDAvis.gensim.prepare(job_description_bow_lda_model, job_description_bow_corpus, job_description_dictionary, sort_topics=True, mds='mmds')
pyLDAvis.save_html(job_description_bow_lda_display, 'job_description_bow_lda.html')
pyLDAvis.display(job_description_bow_lda_display)

In [53]:
job_description_tfidf_lda_display = pyLDAvis.gensim.prepare(job_description_tfidf_lda_model, job_description_tfidf_corpus, job_description_dictionary, sort_topics=True, mds='mmds')
pyLDAvis.save_html(job_description_tfidf_lda_display, 'job_description_tfidf_lda.html')
pyLDAvis.display(job_description_tfidf_lda_display)

In [54]:
job_requirement_bow_lda_display = pyLDAvis.gensim.prepare(job_requirement_bow_lda_model, job_requirement_bow_corpus, job_requirement_dictionary, sort_topics=True, mds='mmds')
pyLDAvis.save_html(job_requirement_bow_lda_display, 'job_requirement_bow_lda.html')
pyLDAvis.display(job_requirement_bow_lda_display)

In [55]:
job_requirement_tfidf_lda_display = pyLDAvis.gensim.prepare(job_requirement_tfidf_lda_model, job_requirement_tfidf_corpus, job_requirement_dictionary, sort_topics=True, mds='mmds')
pyLDAvis.save_html(job_requirement_tfidf_lda_display, 'job_requirement_tfidf_lda.html')
pyLDAvis.display(job_requirement_tfidf_lda_display)

In [56]:
required_qualification_bow_lda_display = pyLDAvis.gensim.prepare(required_qualification_bow_lda_model, required_qualification_bow_corpus, required_qualification_dictionary, sort_topics=True, mds='mmds')
pyLDAvis.save_html(required_qualification_bow_lda_display, 'required_qualification_bow_lda.html')
pyLDAvis.display(required_qualification_bow_lda_display)

In [57]:
required_qualification_tfidf_lda_display = pyLDAvis.gensim.prepare(required_qualification_tfidf_lda_model, required_qualification_tfidf_corpus, required_qualification_dictionary, sort_topics=True, mds='mmds')
pyLDAvis.save_html(required_qualification_tfidf_lda_display, 'required_qualification_tfidf_lda.html')
pyLDAvis.display(required_qualification_tfidf_lda_display)

In [65]:
!ls -ltr

total 2651456
-rw-r--r-- 1 root root   95435519 Apr  3 03:12 online-job-posts.csv
drwxr-xr-x 1 root root       4096 Apr  4 20:20 sample_data
-rw-r--r-- 1 root root  279960406 Apr  7 18:41 test.csv
-rw-r--r-- 1 root root 1251803869 Apr  7 18:41 train.csv
-rw-r--r-- 1 root root  319129989 Apr  7 18:41 user_features.csv
-rw-r--r-- 1 root root   13254376 Apr 22 16:43 online-job-posts.zip
-rw-r--r-- 1 root root  713029252 Apr 22 20:02 dataset.zip
-rw-r--r-- 1 root root    2316149 Apr 22 20:30 job_description_bow_corpus.pkl
-rw-r--r-- 1 root root    2607134 Apr 22 20:30 job_description_tfidf_corpus.pkl
-rw-r--r-- 1 root root     189635 Apr 22 20:30 job_description_dictionary.gensim
-rw-r--r-- 1 root root    5715308 Apr 22 20:30 job_requirement_bow_corpus.pkl
-rw-r--r-- 1 root root    6098902 Apr 22 20:30 job_requirement_tfidf_corpus.pkl
-rw-r--r-- 1 root root     251569 Apr 22 20:30 job_requirement_dictionary.gensim
-rw-r--r-- 1 root root    5714777 Apr 22 20:30 required_qualification_bow_co

In [0]:
files.download("job_description_dictionary.gensim")
files.download("job_description_bow_corpus.pkl")
files.download("job_description_tfidf_corpus.pkl")
files.download("job_description_bow_lda_model.gensim")
files.download("job_description_bow_lda_model.gensim.state")
files.download("job_description_bow_lda_model.gensim.id2word")
files.download("job_description_bow_lda_model.gensim.expElogbeta.npy")
files.download("job_description_tfidf_lda_model.gensim")
files.download("job_description_tfidf_lda_model.gensim.state")
files.download("job_description_tfidf_lda_model.gensim.id2word")
files.download("job_description_tfidf_lda_model.gensim.expElogbeta.npy")

files.download("job_requirement_dictionary.gensim")
files.download("job_requirement_bow_corpus.pkl")
files.download("job_requirement_tfidf_corpus.pkl")
files.download("job_requirement_bow_lda_model.gensim")
files.download("job_requirement_bow_lda_model.gensim.state")
files.download("job_requirement_bow_lda_model.gensim.id2word")
files.download("job_requirement_bow_lda_model.gensim.expElogbeta.npy")
files.download("job_requirement_tfidf_lda_model.gensim")
files.download("job_requirement_tfidf_lda_model.gensim.state")
files.download("job_requirement_tfidf_lda_model.gensim.id2word")
files.download("job_requirement_tfidf_lda_model.gensim.expElogbeta.npy")

files.download("required_qualification_dictionary.gensim")
files.download("required_qualification_bow_corpus.pkl")
files.download("required_qualification_tfidf_corpus.pkl")
files.download("required_qualification_bow_lda_model.gensim")
files.download("required_qualification_bow_lda_model.gensim.state")
files.download("required_qualification_bow_lda_model.gensim.id2word")
files.download("required_qualification_bow_lda_model.gensim.expElogbeta.npy")
files.download("required_qualification_tfidf_lda_model.gensim")
files.download("required_qualification_tfidf_lda_model.gensim.state")
files.download("required_qualification_tfidf_lda_model.gensim.id2word")
files.download("required_qualification_tfidf_lda_model.gensim.expElogbeta.npy")

files.download("job_description_bow_lda.html")
files.download("job_description_tfidf_lda.html")
files.download("job_requirement_bow_lda.html")
files.download("job_requirement_tfidf_lda.html")
files.download("required_qualification_bow_lda.html")
files.download("required_qualification_tfidf_lda.html")